# Modulok

In [7]:
# moduls
import sqlite3
from datetime import datetime, timedelta
import pandas as pd

# Datetime adapter regisztrálása
sqlite3.register_adapter(datetime, lambda dt: dt.isoformat())  # A datetime ISO formátumúra konvertálása

# Kapcsolat az adatbázishoz
conn = sqlite3.connect(r'D:\database\sql_db.db')  
c    = conn.cursor()

# Time Data alap tábla létrehozása

In [2]:
# Tábla törlése, ha már létezik
c.execute('DROP TABLE IF EXISTS time_data')

# Tábla létrehozása: open_time és open_date_time oszlopokkal
c.execute('''
CREATE TABLE IF NOT EXISTS time_data (
    open_time INT UNIQUE,             -- Egyedi kulcs az open_time oszlophoz (timestamp)
    open_date_time TEXT UNIQUE        -- Egyedi kulcs az open_date_time oszlophoz (formázott dátum)
)
''')

# Időbélyegek feltöltése 2017-08-17 06:00:00-tól 2024-12-31 23:59-ig
start_time = datetime(2021, 12, 31, 23, 59)  # Kezdő időpont
end_time   = datetime(2025, 12, 31, 23, 59)  # Befejező időpont

current_time = start_time

# Feltöltés percenkénti időközönként
while current_time <= end_time:
    # Timestamp beszúrása: open_time az int (timestamp)
    open_time = int(current_time.timestamp()) * 1000  # Milliszekundumban tároljuk
    # Dátum és idő formázása: open_date_time a formázott szöveg
    open_date_time = current_time.strftime('%Y-%m-%d %H:%M')  # Formázott dátum és idő (pl. '2024-11-08 12:00')
    
    # Beszúrás az adatbázisba
    c.execute('INSERT OR IGNORE INTO time_data (open_time, open_date_time) VALUES (?, ?)', 
              (open_time, open_date_time))
    current_time += timedelta(minutes=1)  # 1 perces lépés

# Változtatások mentése
conn.commit()

### check data

In [13]:
# Adatbázis kapcsolat létrehozása
pd.read_sql_query(
    """
    SELECT min(open_time), max(open_time), min(open_date_time), max(open_date_time), count(*), count(distinct open_time) 
    FROM time_data
    """, 
conn)

,min(open_time),max(open_time),min(open_date_time),max(open_date_time),count(*),count(distinct open_time)
0,1640991540000,1767221940000,2021-12-31 23:59,2025-12-31 23:59,2103601,2103601


In [8]:
pd.read_sql_query(
""" SELECT * 
    FROM time_data 
    ORDER BY open_time DESC limit 3
""", conn)

,open_time,open_date_time
0,1767221940000,2025-12-31 23:59
1,1767221880000,2025-12-31 23:58
2,1767221820000,2025-12-31 23:57


# FTM alap adat tábla

In [14]:
# Tábla törlése, ha létezik
c.execute('DROP TABLE IF EXISTS ftm_raw_data')

# Tábla definiálása
c.execute('''
CREATE TABLE IF NOT EXISTS ftm_raw_data (
    open_time INT UNIQUE,             -- Egyedi kulcs az open_time oszlophoz
    open_date_time TEXT UNIQUE,       -- Egyedi kulcs az open_date_time oszlophoz
    open FLOAT, 
    high FLOAT, 
    low FLOAT, 
    close FLOAT,
    volume FLOAT
)
''')

conn.commit()

In [15]:
pd.read_sql_query("""select * from ftm_raw_data""",conn)

,open_time,open_date_time,open,high,low,close,volume
